# Problema do Titanic do Kaggle
## ELT 574 - Aprendizado de Máquina
### Especialização em Inteligência Artificial e Computacional

#### Prof. Rodolpho Neves - NIAS/DEL/UFV

#### Problema:
Predizer de pessoas com determinadas características sobreviveriam ao acidente marítimo do Titanic baseado nos dados de sobreviventes do naufrágio.

### Análise do Conjunto de Dados do Titanic

O problema do desastre do **Titanic** é um clássico para quem está começando em **Machine Learning**. O objetivo é prever se um passageiro sobreviveu ou não ao naufrágio com base em algumas características. Dados baixados do Kaggle: [Titanic - Machine Learning from Disaster](https://www.kaggle.com/competitions/titanic).

#### 1. Descrição do Banco de Dados

O conjunto de dados do **Titanic** foi disponibilizado no Kaggle e contém informações sobre os passageiros. Ele possui duas versões principais:
- **Train.csv** (dados de treinamento): usado para treinar o modelo. Possui a informação se o passageiro sobreviveu (`Survived`).
- **Test.csv** (dados de teste): usado para avaliar o modelo. Não possui a coluna `Survived`, e o objetivo é prever esse valor.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### 2. Principais Colunas do Dataset

| Coluna          | Descrição |
|----------------|-------------|
| **PassengerId** | ID único de cada passageiro |
| **Survived**   | Se sobreviveu (1) ou não (0) → [somente no treino] |
| **Pclass**     | Classe do bilhete: 1ª (luxo), 2ª (intermediária), 3ª (econômica) |
| **Name**       | Nome completo do passageiro |
| **Sex**        | Gênero (`male` ou `female`) |
| **Age**        | Idade do passageiro |
| **SibSp**      | Número de irmãos/cônjuge a bordo |
| **Parch**      | Número de pais/filhos a bordo |
| **Ticket**     | Número do bilhete |
| **Fare**       | Valor pago pelo bilhete |
| **Cabin**      | Número da cabine (muitos valores ausentes) |
| **Embarked**   | Porto de embarque (`C` = Cherbourg, `Q` = Queenstown, `S` = Southampton) |

In [2]:
dataset = pd.read_csv('kaggle-data/train.csv')
dataset.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


Aqui, já é possível identificar que algumas variáveis não serão úteis para a resolução do problema, como `PassengerId`, `Name`, `Ticket` e `Cabin`. Estas variáveis podem ser descartadas do banco de dados.

In [3]:
colunas_principais = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = dataset.loc[:, colunas_principais]
y = dataset.loc[:, 'Survived']
X.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S


#### 3. Possíveis Problemas no Dataset
Antes de modelar, precisamos analisar os dados e, se necessário, limpa-los.

In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


As informações do X.info() mostram os tipos das características do banco de dados: float64(2, Age e Fare), int64(4, PassengerId, Pclass, SisSp e Parch), object(5, Name, Sex, Ticket, Cabin e Embarked). Além disso, podemos identificar os seguintes problemas:
- **Valores faltantes:** Muitas idades (`Age`) estão ausentes.
- **Dados categóricos:** `Sex`, `Embarked` e `Pclass` precisam ser convertidos para números. Pode ser feito uma codificação (encoding) com estas variáveis também.
- **Distribuição de classes:** A taxa de sobrevivência não é equilibrada (menos da metade sobreviveu).

Informações numéricas sobre as variáveis inteiras e reais podem ser extraídas com o comando X.describe().

In [5]:
X.describe()

,Pclass,Age,SibSp,Parch,Fare
count,891.000000,714.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.420000,0.000000,0.000000,0.000000
25%,2.000000,20.125000,0.000000,0.000000,7.910400
50%,3.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,38.000000,1.000000,0.000000,31.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200


Podemos analisar os valores médios, máximos, mínimos e os quartis destas variáveis. Entretanto, não conseguimos inferir muita coisa com relação aos passageiros sobreviventes ou não. Se os dados forem agregadas de forma extratificada por variáveis, talvez é possível ter mais ideias sobre interpretações do problema.

In [6]:
# estatisticas dos dados separados por sobreviventes
X.groupby(y).describe().T # transposta

Survived               0           1
Pclass count  549.000000  342.000000
       mean     2.531876    1.950292
       std      0.735805    0.863321
       min      1.000000    1.000000
       25%      2.000000    1.000000
       50%      3.000000    2.000000
       75%      3.000000    3.000000
       max      3.000000    3.000000
Age    count  424.000000  290.000000
       mean    30.626179   28.343690
       std     14.172110   14.950952
       min      1.000000    0.420000
       25%     21.000000   19.000000
       50%     28.000000   28.000000
       75%     39.000000   36.000000
       max     74.000000   80.000000
SibSp  count  549.000000  342.000000
       mean     0.553734    0.473684
       std      1.288399    0.708688
       min      0.000000    0.000000
       25%      0.000000    0.000000
       50%      0.000000    0.000000
       75%      1.000000    1.000000
       max      8.000000    4.000000
Parch  count  549.000000  342.000000
       mean     0.329690    0.464912
       std      0.823166    0.771712
       min      0.000000    0.000000
       25%      0.000000    0.000000
       50%      0.000000    0.000000
       75%      0.000000    1.000000
       max      6.000000    5.000000
Fare   count  549.000000  342.000000
       mean    22.117887   48.395408
       std     31.388207   66.596998
       min      0.000000    0.000000
       25%      7.854200   12.475000
       50%     10.500000   26.000000
       75%     26.000000   57.000000
       max    263.000000  512.329200

In [7]:
# estatisticas dos dados separados por sexo
X.groupby('Sex').describe().T # transposta

Sex               female        male
Pclass count  314.000000  577.000000
       mean     2.159236    2.389948
       std      0.857290    0.813580
       min      1.000000    1.000000
       25%      1.000000    2.000000
       50%      2.000000    3.000000
       75%      3.000000    3.000000
       max      3.000000    3.000000
Age    count  261.000000  453.000000
       mean    27.915709   30.726645
       std     14.110146   14.678201
       min      0.750000    0.420000
       25%     18.000000   21.000000
       50%     27.000000   29.000000
       75%     37.000000   39.000000
       max     63.000000   80.000000
SibSp  count  314.000000  577.000000
       mean     0.694268    0.429809
       std      1.156520    1.061811
       min      0.000000    0.000000
       25%      0.000000    0.000000
       50%      0.000000    0.000000
       75%      1.000000    0.000000
       max      8.000000    8.000000
Parch  count  314.000000  577.000000
       mean     0.649682    0.235702
       std      1.022846    0.612294
       min      0.000000    0.000000
       25%      0.000000    0.000000
       50%      0.000000    0.000000
       75%      1.000000    0.000000
       max      6.000000    5.000000
Fare   count  314.000000  577.000000
       mean    44.479818   25.523893
       std     57.997698   43.138263
       min      6.750000    0.000000
       25%     12.071875    7.895800
       50%     23.000000   10.500000
       75%     55.000000   26.550000
       max    512.329200  512.329200

In [8]:
# estatisticas dos dados separados por classe dos passageiros
X.groupby('Pclass').describe().T # transposta

Pclass                1           2           3
Age   count  186.000000  173.000000  355.000000
      mean    38.233441   29.877630   25.140620
      std     14.802856   14.001077   12.495398
      min      0.920000    0.670000    0.420000
      25%     27.000000   23.000000   18.000000
      50%     37.000000   29.000000   24.000000
      75%     49.000000   36.000000   32.000000
      max     80.000000   70.000000   74.000000
SibSp count  216.000000  184.000000  491.000000
      mean     0.416667    0.402174    0.615071
      std      0.611898    0.601633    1.374883
      min      0.000000    0.000000    0.000000
      25%      0.000000    0.000000    0.000000
      50%      0.000000    0.000000    0.000000
      75%      1.000000    1.000000    1.000000
      max      3.000000    3.000000    8.000000
Parch count  216.000000  184.000000  491.000000
      mean     0.356481    0.380435    0.393075
      std      0.693997    0.690963    0.888861
      min      0.000000    0.000000    0.000000
      25%      0.000000    0.000000    0.000000
      50%      0.000000    0.000000    0.000000
      75%      0.000000    1.000000    0.000000
      max      4.000000    3.000000    6.000000
Fare  count  216.000000  184.000000  491.000000
      mean    84.154687   20.662183   13.675550
      std     78.380373   13.417399   11.778142
      min      0.000000    0.000000    0.000000
      25%     30.923950   13.000000    7.750000
      50%     60.287500   14.250000    8.050000
      75%     93.500000   26.000000   15.500000
      max    512.329200   73.500000   69.550000

In [9]:
# estatisticas dos dados separados por classe dos passageiros
X.groupby('Embarked').describe().T # transposta

Embarked               C          Q           S
Pclass count  168.000000  77.000000  644.000000
       mean     1.886905   2.909091    2.350932
       std      0.944100   0.369274    0.789402
       min      1.000000   1.000000    1.000000
       25%      1.000000   3.000000    2.000000
       50%      1.000000   3.000000    3.000000
       75%      3.000000   3.000000    3.000000
       max      3.000000   3.000000    3.000000
Age    count  130.000000  28.000000  554.000000
       mean    30.814769  28.089286   29.445397
       std     15.434860  16.915396   14.143192
       min      0.420000   2.000000    0.670000
       25%     21.250000  17.500000   21.000000
       50%     29.000000  27.000000   28.000000
       75%     40.000000  34.500000   38.000000
       max     71.000000  70.500000   80.000000
SibSp  count  168.000000  77.000000  644.000000
       mean     0.386905   0.428571    0.571429
       std      0.557213   0.979105    1.216600
       min      0.000000   0.000000    0.000000
       25%      0.000000   0.000000    0.000000
       50%      0.000000   0.000000    0.000000
       75%      1.000000   0.000000    1.000000
       max      2.000000   4.000000    8.000000
Parch  count  168.000000  77.000000  644.000000
       mean     0.363095   0.168831    0.413043
       std      0.660481   0.656999    0.853253
       min      0.000000   0.000000    0.000000
       25%      0.000000   0.000000    0.000000
       50%      0.000000   0.000000    0.000000
       75%      1.000000   0.000000    0.000000
       max      3.000000   5.000000    6.000000
Fare   count  168.000000  77.000000  644.000000
       mean    59.954144  13.276030   27.079812
       std     83.912994  14.188047   35.887993
       min      4.012500   6.750000    0.000000
       25%     13.697950   7.750000    8.050000
       50%     29.700000   7.750000   13.000000
       75%     78.500025  15.500000   27.900000
       max    512.329200  90.000000  263.000000

De todas estas métricas, as únicas que tem uma diferença considerável entre as médias é quando extratificamos em relação às classes dos passageiros. Estas informações podem ser úteis caso tenhamos que inputar dados posteriormente.